# INFORME TÉCNICO: VRP Multi-Trip (Flota de 2 Vehículos)

## 1. Definición del Escenario

En esta versión, se modela una operativa realista donde la flota es limitada, pero el tiempo no. 

**Parámetros:**
* **Capacidad:** 80 kg (Estándar).
* **Flota Física:** 2 Vehículos.
* **Operativa:** Los vehículos pueden retornar al depósito, recargar y realizar un segundo viaje.

**Lógica de Solución:**
El Algoritmo Genético calculará los viajes necesarios (rutas lógicas) para atender toda la demanda. Posteriormente, un algoritmo de asignación distribuirá esos viajes entre los 2 vehículos disponibles para equilibrar la carga de trabajo.

In [ ]:
import random
import math
import matplotlib.pyplot as plt

# Datos
depot = [20, 120]
clientes = [
    [35, 115], [50, 140], [70, 100], [40, 80], [25, 60]
]
pesos = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]
pedidos = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

capacidad = 80
n_vehiculos_fisicos = 2

In [ ]:
# Cálculo de Demandas
demandas = []
for pedido in pedidos:
    total = 0
    for item, cant in pedido:
        total += pesos[item - 1] * cant
    demandas.append(total)

clientes_validos = []
demandas_validas = []
for i in range(len(clientes)):
    if demandas[i] <= capacidad:
        clientes_validos.append(clientes[i])
        demandas_validas.append(demandas[i])

print(f"Demanda Total a servir: {sum(demandas_validas)} kg")

In [ ]:
# --- Lógica del AG (Idéntica a la versión base) ---
# El AG busca minimizar la distancia total de los viajes necesarios.
# Luego nosotros asignaremos esos viajes a los coches.

def distancia(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def dividir_en_rutas(ruta):
    rutas = []
    carga = 0
    actual = []
    for c in ruta:
        if carga + demandas_validas[c] <= capacidad:
            actual.append(c)
            carga += demandas_validas[c]
        else:
            rutas.append(actual)
            actual = [c]
            carga = demandas_validas[c]
    if actual:
        rutas.append(actual)
    return rutas

def distancia_total(ruta):
    rutas = dividir_en_rutas(ruta)
    total = 0
    for r in rutas:
        pos = depot
        for c in r:
            total += distancia(pos, clientes_validos[c])
            pos = clientes_validos[c]
        total += distancia(pos, depot)
    return total

def fitness(ruta):
    return distancia_total(ruta)

def crear_poblacion(n, n_clientes):
    poblacion = []
    base = list(range(n_clientes))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        poblacion.append(r)
    return poblacion

def seleccion(poblacion, fitnesses):
    i1, i2 = random.sample(range(len(poblacion)), 2)
    return poblacion[i1][:] if fitnesses[i1] < fitnesses[i2] else poblacion[i2][:]

def cruce(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    hijo = [-1] * len(p1)
    hijo[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in hijo:
            if pos >= len(p1): pos = 0
            hijo[pos] = x
            pos += 1
    return hijo

def mutacion(ruta, prob=0.1):
    if random.random() < prob:
        i, j = random.sample(range(len(ruta)), 2)
        ruta[i], ruta[j] = ruta[j], ruta[i]

def algoritmo_genetico(n_generaciones=200, tam_pob=50):
    poblacion = crear_poblacion(tam_pob, len(clientes_validos))
    fitnesses = [fitness(r) for r in poblacion]

    for _ in range(n_generaciones):
        nueva = []
        for _ in range(tam_pob):
            p1 = seleccion(poblacion, fitnesses)
            p2 = seleccion(poblacion, fitnesses)
            hijo = cruce(p1, p2)
            mutacion(hijo, 0.1)
            nueva.append(hijo)
        poblacion = nueva
        fitnesses = [fitness(r) for r in poblacion]

    mejor = min(range(tam_pob), key=lambda i: fitnesses[i])
    return poblacion[mejor], fitnesses[mejor]

In [ ]:
# Ejecutar AG para obtener las rutas lógicas necesarias
mejor_ruta, mejor_valor = algoritmo_genetico()
viajes_necesarios = dividir_en_rutas(mejor_ruta)

print(f"Viajes totales necesarios: {len(viajes_necesarios)}")

# --- ASIGNACIÓN DE VIAJES A VEHÍCULOS FÍSICOS ---
# Asignación simple 'Round Robin' para balancear
vehiculos_asignados = {0: [], 1: []} # Vehiculo 0 y Vehiculo 1

for i, viaje in enumerate(viajes_necesarios):
    id_vehiculo = i % n_vehiculos_fisicos
    vehiculos_asignados[id_vehiculo].append(viaje)

print("\n--- Planificación de Flota ---")
for v_id, viajes in vehiculos_asignados.items():
    print(f"VEHÍCULO FÍSICO {v_id + 1}:")
    if not viajes:
        print("  (Sin actividad)")
    for idx, r in enumerate(viajes):
        carga = sum(demandas_validas[c] for c in r)
        print(f"  -> Turno {idx+1}: Clientes {r} | Carga: {round(carga,1)} kg")

In [ ]:
def plot_multitrip(vehiculos_asignados):
    plt.figure(figsize=(9, 9))
    
    # Depósito
    plt.scatter(depot[0], depot[1], c='red', s=150, marker='s', label='Depósito', zorder=10)
    
    # Clientes
    for i, (x, y) in enumerate(clientes_validos):
        plt.scatter(x, y, c='blue', zorder=5)
        plt.text(x+1, y+1, f'C{i+1}', fontsize=9)

    colores_vehiculo = ['#1f77b4', '#ff7f0e'] # Azul y Naranja
    estilos_linea = ['-', '--', ':'] # Sólido, Rayas, Puntos (para distinguir turnos)
    
    for v_id, viajes in vehiculos_asignados.items():
        color = colores_vehiculo[v_id]
        
        for idx_viaje, r in enumerate(viajes):
            estilo = estilos_linea[idx_viaje % len(estilos_linea)]
            
            puntos = [depot] + [clientes_validos[c] for c in r] + [depot]
            xs = [p[0] for p in puntos]
            ys = [p[1] for p in puntos]
            
            label = f'V{v_id+1} - Turno {idx_viaje+1}'
            plt.plot(xs, ys, linestyle=estilo, color=color, linewidth=2, label=label, alpha=0.8)
            
            # Anotar carga en el centro de la ruta
            mid = len(xs) // 2
            plt.text(xs[mid], ys[mid], f'V{v_id+1}.{idx_viaje+1}', 
                     color=color, fontweight='bold', bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

    plt.title("Operativa Multi-Trip (2 Vehículos)")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

plot_multitrip(vehiculos_asignados)

## 5. Conclusiones Finales (Escenario Multi-Trip)

Al permitir que los vehículos realicen múltiples viajes (recarga en depósito), se logra satisfacer la demanda completa bajo las restricciones originales (Capacidad 80 kg y 2 Vehículos):

1.  **Operativa de Doble Turno:**
    * El análisis de demanda (227.1 kg) determina que son necesarios **3 viajes** en total.
    * Con 2 vehículos disponibles, la solución óptima implica una asimetría en la carga de trabajo:
        * **Vehículo 1:** Realiza **2 Turnos**. (Ej: Un viaje pesado con el Cliente 5, retorno, y un segundo viaje ligero).
        * **Vehículo 2:** Realiza **1 Turno**.

2.  **Eficiencia de Activos:**
    * Esta estrategia evita la inversión en un tercer camión o en camiones más grandes.
    * El costo a pagar no es monetario (inversión), sino temporal: el tiempo total de operación se extiende porque un vehículo debe esperar a volver al depósito para iniciar su segunda ruta.

3.  **Flexibilidad:**
    * El modelo demuestra robustez. Aunque la capacidad de 80 kg es limitante (fragmenta mucho los pedidos), la política de "retorno y recarga" permite completar el servicio al 100% sin dejar clientes desatendidos.